In [1]:
from openeye import oechem

def mol_from_freesolv_id(freesolv_id):
    fname = '../../../MSKCC/Chodera Lab/feedstock/FreeSolv-0.51/mol2files_gaff/{}.mol2'.format(freesolv_id)

    ifs = oechem.oemolistream()

    if ifs.open(fname):
        for mol_ in ifs.GetOEMols():
            mol = oechem.OEMol(mol_)
    return mol

In [2]:
from bayes_implicit_solvent.solvation_free_energy import db, smiles_list, mol_top_sys_pos_list

In [3]:
db[0]

['mobley_1017962',
 'CCCCCC(=O)OC',
 'methyl hexanoate',
 '-2.49',
 '0.60',
 '-3.30',
 '0.03',
 '10.1021/ct050097l',
 '10.1021/ct800409d',
 'Experimental uncertainty not presently available, so assigned a default value.  ']

In [4]:
smiles_to_freesolv_id_dict = {}
for i in range(len(db)):
    smiles_to_freesolv_id_dict[db[i][1]] = db[i][0]

In [5]:
freesolv_mols = []

for smiles in smiles_list:
    freesolv_mols.append(mol_from_freesolv_id(smiles_to_freesolv_id_dict[smiles]))

In [9]:
mol = freesolv_mols[0]
len(list(mol.GetAtoms()))

27

In [24]:
okay = []

discrepancies = []
for i in range(len(freesolv_mols)):
    
    mol2_n_atoms = len(list(freesolv_mols[i].GetAtoms()))
    repr_n_atoms = len(list(mol_top_sys_pos_list[i][0].GetAtoms()))
    
    if mol2_n_atoms != repr_n_atoms:
        okay.append(False)
        discrepancies.append((mol2_n_atoms, repr_n_atoms))
        print(smiles_to_freesolv_id_dict[smiles_list[i]])
        print('\tFreeSolvs mol2 file had {} atoms'.format(mol2_n_atoms))
        print('\tinput file reproduced using OpenEye had {} atoms'.format(repr_n_atoms))
    else:
        okay.append(True)

mobley_3274817
	FreeSolvs mol2 file had 29 atoms
	input file reproduced using OpenEye had 28 atoms
mobley_2099370
	FreeSolvs mol2 file had 33 atoms
	input file reproduced using OpenEye had 32 atoms
mobley_1527293
	FreeSolvs mol2 file had 31 atoms
	input file reproduced using OpenEye had 30 atoms
mobley_4371692
	FreeSolvs mol2 file had 27 atoms
	input file reproduced using OpenEye had 26 atoms
mobley_2269032
	FreeSolvs mol2 file had 31 atoms
	input file reproduced using OpenEye had 30 atoms
mobley_2078467
	FreeSolvs mol2 file had 33 atoms
	input file reproduced using OpenEye had 32 atoms
mobley_6055410
	FreeSolvs mol2 file had 26 atoms
	input file reproduced using OpenEye had 25 atoms
mobley_5282042
	FreeSolvs mol2 file had 44 atoms
	input file reproduced using OpenEye had 45 atoms
mobley_4936555
	FreeSolvs mol2 file had 33 atoms
	input file reproduced using OpenEye had 32 atoms
mobley_1944394
	FreeSolvs mol2 file had 40 atoms
	input file reproduced using OpenEye had 41 atoms
mobley_506

In [25]:
n_discrepancies

48

In [26]:
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
sum([a > b for (a,b) in discrepancies]) / len(discrepancies)

0.4166666666666667

In [28]:
# they're all just off by one...
sum([abs(a - b) > 1 for (a,b) in discrepancies]) / len(discrepancies)

0.0

In [32]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from openeye import oeomega, oechem, oequacpac
from openforcefield.typing.engines.smirnoff import ForceField, generateTopologyFromOEMol
from openmmtools.constants import kB
from openmoltools import utils
from openmoltools.forcefield_generators import generateResidueTemplate, generateTopologyFromOEMol, \
    gaffTemplateGenerator
from scipy.stats import entropy
from simtk import unit
from simtk.openmm.app import ForceField

ff = ForceField('smirnoff99Frosst.offxml')

omega = oeomega.OEOmega()
omega.SetMaxConfs(800)
omega.SetIncludeInput(False)
omega.SetStrictStereo(True)


def generate_oemol(smiles):
    """Add hydrogens, assign charges, generate conformers, and return molecule"""
    mol = oechem.OEMol()
    oechem.OEAssignAromaticFlags(mol, oechem.OEAroModelOpenEye)
    chargeEngine = oequacpac.OEAM1BCCCharges()
    oechem.OEParseSmiles(mol, smiles)
    oechem.OEAddExplicitHydrogens(mol)
    status = omega(mol)
    if not status: print("Something went wrong in `generate_oemol({})!".format(smiles))
    oechem.OETriposAtomNames(mol)
    oequacpac.OESetNeutralpHModel(mol)
    oequacpac.OEAssignCharges(mol, chargeEngine)
    _ = generateTopologyFromOEMol(mol)
    return mol

def create_gaff_system(mol, gaff_version='gaff'):
    # create force field
    gaff_xml_filename = utils.get_data_filename("parameters/{}.xml".format(gaff_version))
    forcefield = ForceField(gaff_xml_filename)
    forcefield.registerTemplateGenerator(gaffTemplateGenerator)

    # register template for molecule
    template, additional_parameters_ffxml = generateResidueTemplate(mol, gaff_version=gaff_version)
    forcefield.registerResidueTemplate(template)

    # create simulation
    topology = generateTopologyFromOEMol(mol)
    system = forcefield.createSystem(topology)
    return system


def get_torsion_dict(torsion_force):
    torsions = defaultdict(list)
    for i in range(torsion_force.getNumTorsions()):
        a, b, c, d, periodicity, phase, force_constant = torsion_force.getTorsionParameters(i)
        # make sure we can compare torsion tuple equality straightforwardly later
        if d < a:
            (a, b, c, d) = (d, c, b, a)
            phase = - phase
        torsions[(a, b, c, d)].append({'periodicity': periodicity,
                                       'phase': phase,
                                       'force_constant': force_constant,
                                       })
    return torsions


kT = kB * 298.15 * unit.kelvin

theta = np.linspace(-np.pi, np.pi, 1000) * unit.radian

def evaluate_torsion(theta, periodicity=1, phase=0, force_constant=1):
    return force_constant * (1 + np.cos(periodicity * theta - phase))


def compute_full_torsion(theta, torsion_list):
    U = evaluate_torsion(theta, **torsion_list[0])
    for torsion in torsion_list[1:]:
        U += evaluate_torsion(theta, **torsion)
    return U


def compute_torsion_distribution(theta, torsion_list):
    U = compute_full_torsion(theta, torsion_list)
    q = np.exp(-U / kT)
    Z = np.trapz(q, theta)
    p = q / Z
    return p

def describe_torsion(torsion_tuple):
    atom_names = [a.name for a in topology.atoms()]
    return ' - '.join(atom_names[i] for i in torsion_tuple)


for i in [i for i in range(len(mol_top_sys_pos_list)) if okay[i]]:
    mol, topology, smirnoff_system, pos = mol_top_sys_pos_list[i]
    freesolv_id = smiles_to_freesolv_id_dict[smiles_list[i]]
    print(freesolv_id)
    
    gaff_system = create_gaff_system(mol, gaff_version='gaff')

    torsion_force_gaff = gaff_system.getForce(2)
    torsion_force_smirnoff = smirnoff_system.getForce(2)
    smirnoff_torsions = get_torsion_dict(torsion_force_smirnoff)
    gaff_torsions = get_torsion_dict(torsion_force_gaff)

    # identify all the 4-tuples for this molecule that have a torsion term in gaff and a torsion term in smirnoff
    torsions_in_common = sorted(list(set(gaff_torsions.keys()).intersection(smirnoff_torsions.keys())))

    # compute the implied torsion distributions for each torsion under gaff and smirnoff
    torsion_distributions = []
    for torsion_tuple in torsions_in_common:
        p_gaff = compute_torsion_distribution(
            theta, gaff_torsions[torsion_tuple])

        p_smirnoff = compute_torsion_distribution(
            theta, smirnoff_torsions[torsion_tuple])

        torsion_distributions.append((p_gaff, p_smirnoff))

    # only interested in the ones where they disagree a bit
    relative_entropies = [entropy(p_gaff, p_smirnoff) for (p_gaff, p_smirnoff) in torsion_distributions]
    interesting_torsions = [torsion_tuple for (i, torsion_tuple) in enumerate(torsions_in_common) if
                            relative_entropies[i] > 1e-2]

    print('\t# torsions in common: {}'.format(len(torsions_in_common)))
    print('\t# interesting torsions: {}'.format(len(interesting_torsions)))

    # plot result
    
    if len(interesting_torsions) > 0:
        n_rows = len(interesting_torsions)
        n_cols = 2
        plt.figure(figsize=(4 * n_cols, 4 * n_rows))

        i = 0
        for torsion_tuple in interesting_torsions:
            # energies
            i += 1
            ax = plt.subplot(n_rows, n_cols, i)
            plt.title('torsion potential\n' + describe_torsion(torsion_tuple))

            plot_kwargs = {}

            # plot smirnoff energies
            plt.plot(theta,
                     compute_full_torsion(
                         theta, smirnoff_torsions[torsion_tuple]).value_in_unit(
                         unit.kilojoule_per_mole),
                     label='SMIRNOFF', **plot_kwargs)

            # plot gaff energies
            plt.plot(theta,
                     compute_full_torsion(
                         theta, gaff_torsions[torsion_tuple]).value_in_unit(
                         unit.kilojoule_per_mole),
                     label='GAFF', **plot_kwargs)

            plt.xlim(-np.pi, np.pi)
            plt.xticks([-np.pi, 0, np.pi], [r'$-\pi$', '0', r'$\pi$'])
            plt.xlabel(r'torsion angle $\theta$')

            plt.ylim(0, )
            plt.ylabel(r'$U(\theta)$ (in kJ/mol)')

            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

            # distributions
            i += 1
            ax = plt.subplot(n_rows, n_cols, i)
            plt.title('torsion distribution (independent)\n' + describe_torsion(torsion_tuple))

            plot_kwargs = {}
            fill_kwargs = {'alpha': 0.2}

            # plot smirnoff implied equilibrium distribution
            plt.plot(theta,
                     compute_torsion_distribution(
                         theta, smirnoff_torsions[torsion_tuple]),
                     label='SMIRNOFF', **plot_kwargs)
            plt.fill_between(theta,
                             compute_torsion_distribution(
                                 theta, smirnoff_torsions[torsion_tuple]), **fill_kwargs)

            # plot gaff implied equilibrium distribution
            plt.plot(theta,
                     compute_torsion_distribution(
                         theta, gaff_torsions[torsion_tuple]),
                     label='GAFF')
            plt.fill_between(theta,
                             compute_torsion_distribution(
                                 theta, gaff_torsions[torsion_tuple]), **fill_kwargs)

            plt.legend(title='force field', loc='upper right')

            plt.xlim(-np.pi, np.pi)
            plt.xticks([-np.pi, 0, np.pi], [r'$-\pi$', '0', r'$\pi$'])
            plt.xlabel(r'torsion angle $\theta$')

            plt.ylim(0, )
            plt.yticks([])
            plt.ylabel(r'$p(\theta)$ at $T=$298.15K')

            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

        plt.tight_layout()
        plt.savefig('{}-torsions-comparison-gaff.png'.format(freesolv_id), bbox_inches='tight')
        plt.close()

mobley_8117218
	# torsions in common: 93
	# interesting torsions: 17
mobley_6733657
	# torsions in common: 85
	# interesting torsions: 3
mobley_468867
	# torsions in common: 63
	# interesting torsions: 7
mobley_9571888
	# torsions in common: 77
	# interesting torsions: 7
mobley_2501588
	# torsions in common: 81
	# interesting torsions: 39
mobley_7829570
	# torsions in common: 75
	# interesting torsions: 15
mobley_2725215
	# torsions in common: 89
	# interesting torsions: 29
mobley_5456566
	# torsions in common: 58
	# interesting torsions: 10
mobley_5076071
	# torsions in common: 60
	# interesting torsions: 18
mobley_7176248
	# torsions in common: 75
	# interesting torsions: 15
mobley_9821936
	# torsions in common: 64
	# interesting torsions: 8
mobley_9534740
	# torsions in common: 66
	# interesting torsions: 2
mobley_8208692
	# torsions in common: 59
	# interesting torsions: 11
mobley_2960202
	# torsions in common: 64
	# interesting torsions: 8
mobley_1417007
	# torsions in common: 56


mobley_4479135
	# torsions in common: 62
	# interesting torsions: 14
mobley_7912193
	# torsions in common: 67
	# interesting torsions: 9
mobley_7364468
	# torsions in common: 47
	# interesting torsions: 11
mobley_9897248
	# torsions in common: 33
	# interesting torsions: 17
mobley_8207196
	# torsions in common: 38
	# interesting torsions: 20
mobley_4715906
	# torsions in common: 72
	# interesting torsions: 0
mobley_2492140
	# torsions in common: 9
	# interesting torsions: 3
mobley_5494918
	# torsions in common: 4
	# interesting torsions: 4
mobley_4964807
	# torsions in common: 11
	# interesting torsions: 8
mobley_9717937
	# torsions in common: 61
	# interesting torsions: 13
mobley_8785107
	# torsions in common: 51
	# interesting torsions: 0
mobley_819018
	# torsions in common: 33
	# interesting torsions: 17
mobley_3980099
	# torsions in common: 30
	# interesting torsions: 6
mobley_2261979
	# torsions in common: 61
	# interesting torsions: 8
mobley_6201330
	# torsions in common: 58
	# i

mobley_7200804
	# torsions in common: 31
	# interesting torsions: 5
mobley_4553008
	# torsions in common: 30
	# interesting torsions: 6
mobley_4177472
	# torsions in common: 63
	# interesting torsions: 0
mobley_7608435
	# torsions in common: 62
	# interesting torsions: 3
mobley_6175884
	# torsions in common: 34
	# interesting torsions: 10
mobley_1139153
	# torsions in common: 63
	# interesting torsions: 0
mobley_3746675
	# torsions in common: 33
	# interesting torsions: 9
mobley_282648
	# torsions in common: 52
	# interesting torsions: 8
mobley_8789465
	# torsions in common: 31
	# interesting torsions: 5
mobley_6794076
	# torsions in common: 30
	# interesting torsions: 12
mobley_1046331
	# torsions in common: 34
	# interesting torsions: 7
mobley_4699732
	# torsions in common: 18
	# interesting torsions: 0
mobley_5747981
	# torsions in common: 33
	# interesting torsions: 0
mobley_2043882
	# torsions in common: 59
	# interesting torsions: 5
mobley_5471704
	# torsions in common: 45
	# int

	# torsions in common: 27
	# interesting torsions: 9
mobley_8798016
	# torsions in common: 27
	# interesting torsions: 9
mobley_8436428
	# torsions in common: 54
	# interesting torsions: 0
mobley_1760914
	# torsions in common: 30
	# interesting torsions: 6
mobley_20524
	# torsions in common: 31
	# interesting torsions: 5
mobley_5026370
	# torsions in common: 22
	# interesting torsions: 4
mobley_9974966
	# torsions in common: 42
	# interesting torsions: 4
mobley_1708457
	# torsions in common: 18
	# interesting torsions: 18
mobley_7010316
	# torsions in common: 57
	# interesting torsions: 0
mobley_6430250
	# torsions in common: 48
	# interesting torsions: 6
mobley_4603202
	# torsions in common: 24
	# interesting torsions: 0
mobley_3398536
	# torsions in common: 30
	# interesting torsions: 6
mobley_7203421
	# torsions in common: 45
	# interesting torsions: 6
mobley_9114381
	# torsions in common: 8
	# interesting torsions: 3
mobley_252413
	# torsions in common: 54
	# interesting torsions: 

	# torsions in common: 9
	# interesting torsions: 8
mobley_4762983
	# torsions in common: 42
	# interesting torsions: 6
mobley_5690766
	# torsions in common: 21
	# interesting torsions: 12
mobley_7227357
	# torsions in common: 39
	# interesting torsions: 0
mobley_1857976
	# torsions in common: 9
	# interesting torsions: 0
mobley_3982371
	# torsions in common: 9
	# interesting torsions: 4
mobley_9705941
	# torsions in common: 12
	# interesting torsions: 12
mobley_2143011
	# torsions in common: 23
	# interesting torsions: 5
mobley_1827204
	# torsions in common: 39
	# interesting torsions: 3
mobley_3775790
	# torsions in common: 20
	# interesting torsions: 20
mobley_3738859
	# torsions in common: 36
	# interesting torsions: 0
mobley_8614858
	# torsions in common: 54
	# interesting torsions: 0
mobley_8492526
	# torsions in common: 30
	# interesting torsions: 3
mobley_7542832
	# torsions in common: 42
	# interesting torsions: 0
mobley_8578590
	# torsions in common: 12
	# interesting torsion